In [31]:
# import logging
# logging.getLogger().setLevel(logging.INFO)
import apache_beam as beam
import pandas as pd
import pyarrow as pa

def get_schema(pcoll):
    df = pd.DataFrame(pcoll)
    tbl = pa.Table.from_pandas(df)
    schema = tbl.schema
    yield schema
    
def write_parquet(pcoll, schema):
    yield pcoll \
        | beam.io.WriteToParquet('test.parquet', schema=schema)

with beam.Pipeline() as pipeline:
    
    pc = pipeline \
        | beam.Create([
            {'a': 'test', 'b': 1},
            {'a': 'live', 'b': 0}
        ])
        
    schema = pc \
        | beam.combiners.Sample.FixedSizeGlobally(1) \
        | beam.FlatMap(get_schema)
    
    pc \
        | beam.FlatMap(write_parquet,
            schema=beam.pvalue.AsSingleton(schema)) \
#         | beam.Map(print)
    
#     schema | beam.Map(print)

TypeError: PTransform Create: Refusing to treat string as an iterable. (string='test') [while running '[31]: FlatMap(write_parquet)']